In [1]:
%run 00_display_tables.ipynb

Successfully connected to py_brew database.
+-------------+-------------------+---------------+----------------+
| customer_id |   customer_name   | customer_city | customer_state |
+-------------+-------------------+---------------+----------------+
|     100     | Jennifer Gonzalez |   Smithland   |    Arkansas    |
+-------------+-------------------+---------------+----------------+
+---------+---------------------+------------+-------------+-------+------+
| date_id |        tmstp        |    date    | day_of_week | month | year |
+---------+---------------------+------------+-------------+-------+------+
|    1    | 2020-07-04 21:23:10 | 2020-07-04 |      4      |   7   | 2020 |
+---------+---------------------+------------+-------------+-------+------+
+------------------+------------------+---------------------+--------------+---------------+
|    product_id    | product_category | product_subcategory | product_name | product_price |
+------------------+------------------+------

#### Determine the products that have experienced a significant increase in sales amount compared to the previous month. Consider a significant increase to be a growth rate of at least ***200%***. Include the product information and the corresponding months.

##### ***Hint: Combine window functions, filtering conditions, and calculations to identify products with a significant increase in sales amount compared to the previous month. Apply the growth rate threshold.***

In [27]:
%%sql 

WITH current_month_CTE AS (
    SELECT 
        p.product_id,
        p.product_name,
        MONTHNAME(d.date) AS current_sale_month,
        SUM(f.sales_amount) AS current_month_sale_total
    FROM fact_sales f 
    JOIN dim_date d ON f.date_id = d.date_id
    JOIN dim_product p ON f.product_id = p.product_id
    WHERE d.date >= DATE_SUB(CURRENT_DATE(), INTERVAL 2 MONTH)
    GROUP BY p.product_id, p.product_name, MONTHNAME(d.date)
), previous_month_CTE AS(
    SELECT
        p.product_id,
        p.product_name,
        MONTHNAME(d.date) AS previous_sale_month,
        SUM(f.sales_amount) AS previous_month_sale_total
    FROM fact_sales f
    JOIN dim_date d ON f.date_id = d.date_id
    JOIN dim_product p ON f.product_id = p.product_id
    WHERE 
        d.date >= DATE_SUB(CURRENT_DATE(), INTERVAL 3 MONTH)
        AND d.date < DATE_SUB(CURRENT_DATE(), INTERVAL 1 MONTH)
    GROUP BY p.product_id, p.product_name, MONTHNAME(d.date)
) 
SELECT 
    curr.product_name,
    current_sale_month,
    previous_sale_month,
    current_month_sale_total,
    previous_month_sale_total,
    (current_month_sale_total - previous_month_sale_total) / previous_month_sale_total AS growth_rate
FROM current_month_CTE AS curr 
JOIN previous_month_CTE AS prev ON curr.product_id = prev.product_id
WHERE (current_month_sale_total - previous_month_sale_total) / previous_month_sale_total >= 2
ORDER BY growth_rate DESC;


product_name,current_sale_month,previous_sale_month,current_month_sale_total,previous_month_sale_total,growth_rate
Black Tea,March,February,67.01,9.13,6.339540
Iced Earl Grey Tea,April,April,34.78,5.08,5.846457
Cookie,April,April,38.38,6.52,4.886503
Iced Chai Tea,May,April,46.61,8.93,4.219485
Iced Chai Tea,March,April,45.80,8.93,4.128779
Cinammon Roll,April,March,30.69,6.00,4.115000
Iced Black Tea,May,February,37.48,7.67,3.886571
Espresso,April,April,32.17,7.08,3.543785
Black Tea,April,April,38.51,8.62,3.467517
Espresso,May,March,40.01,9.09,3.401540
